In [5]:
!pip install simpletransformers

In [6]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

In [8]:
data=pd.read_csv('train.csv')
data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20972 entries, 0 to 20971
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    20972 non-null  int64 
 1   TITLE                 20972 non-null  object
 2   ABSTRACT              20972 non-null  object
 3   Computer Science      20972 non-null  int64 
 4   Physics               20972 non-null  int64 
 5   Mathematics           20972 non-null  int64 
 6   Statistics            20972 non-null  int64 
 7   Quantitative Biology  20972 non-null  int64 
 8   Quantitative Finance  20972 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.4+ MB


In [10]:
data.isnull().sum()

,0
ID,0
TITLE,0
ABSTRACT,0
Computer Science,0
Physics,0
Mathematics,0
Statistics,0
Quantitative Biology,0
Quantitative Finance,0


In [11]:
data.drop(columns='ID', inplace=True)

In [12]:
data['text']=data['TITLE']+ '  '+ data['ABSTRACT']

In [13]:
data['text'].head()

,text
0,Reconstructing Subject-Specific Effect Maps ...
1,Rotation Invariance Neural Network Rotation...
2,Spherical polyharmonics and Poisson kernels fo...
3,A finite element approximation for the stochas...
4,Comparative study of Discrete Wavelet Transfor...


In [14]:
data.rename(columns={'Computer Science': 'Computer_Science','Quantitative Biology': 'Quantitative_Biology', 'Quantitative Finance': 'Quantitative_Finance'}, inplace=True)

In [15]:

data['labels'] = list(zip(data.Computer_Science.tolist(), data.Physics.tolist(), data.Mathematics.tolist(), data.Statistics.tolist(),  data.Quantitative_Biology.tolist(), data.Quantitative_Finance.tolist()))

In [16]:
data['labels'].head()

,labels
0,"(1, 0, 0, 0, 0, 0)"
1,"(1, 0, 0, 0, 0, 0)"
2,"(0, 0, 1, 0, 0, 0)"
3,"(0, 0, 1, 0, 0, 0)"
4,"(1, 0, 0, 1, 0, 0)"


In [17]:
train_df, eval_df=train_test_split(data[['text', 'labels']], test_size=0.2, random_state=42)

In [18]:
train_df.columns=['text', 'labels']
eval_df.columns=['text', 'labels']

In [19]:
model_args = MultiLabelClassificationArgs(num_train_epochs=1, output_dir='./fine_tuned')

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=6,
    use_cuda=True,
    attn_implementation="eager",
    args=model_args,
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/2098 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/8 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/42 [00:00<?, ?it/s]

In [22]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Machine Learning and Data Science"])

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
predictions[0]

subjects=['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']

for i in range(6):
  if predictions[0][i]==1:
    print(subjects[i])

Computer Science
Statistics


In [33]:
input_text="""

The interconnectedness of various scientific and mathematical disciplines drives innovation
 and understanding across diverse fields. Computational models help predict natural phenomena,
 while mathematical frameworks underpin theoretical advancements and practical applications. Analytical
 methods enable the extraction of meaningful insights from complex data, leading to breakthroughs in both
 the natural world and financial systems. The application of quantitative techniques to biological
 processes and market behaviors reveals underlying patterns, enhancing our ability to make informed
decisions. This synergy of knowledge fosters progress, allowing us to solve intricate problems
and advance our understanding of the world.
"""

input_text2="""

the foundation of logical reasoning and problem-solving,
serving as the universal language across disciplines. It provides the tools to
quantify and understand the world, from the simplest patterns to the most complex systems.
 Through abstraction and generalization, mathematics reveals underlying structures and relationships,
 enabling us to predict and explain phenomena in science, engineering, economics, and beyond.
 It fosters critical thinking and precision, encouraging us to approach problems systematically and
 creatively. Whether through the elegance of pure theory or the practicality of applied methods,
mathematics continues to be a driving force in human progress and discovery.

"""

input_text3="""

By quantifying biological processes, researchers can analyze complex interactions,
such as gene expression, cellular dynamics, and ecological relationships.
This approach allows for precise predictions and the identification of
patterns that might be missed through traditional observational methods.

"""

new_predictions, raw_outputs=model.predict([input_text, input_text2, input_text3])

print(new_predictions)

for i in range(6):
  for j in range(len(new_predictions)):
    if new_predictions[j][i]==1:
      print(subjects[i])

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
Computer Science
Computer Science
Quantitative Biology
